## Mapping

In [1]:
import torch
import os
import sys
base_dir = os.environ.get('TORCHSIM_DIR', default='/workspace/PyTorchSim')
sys.path.append(base_dir)

### Default mapping is based on heuristic.

In [2]:
from Scheduler.scheduler import PyTorchSimRunner
device = PyTorchSimRunner.setup_device().custom_device()

input = torch.randn(1024, 1024).to(device=device)
weight = torch.randn(1024, 1024).to(device=device)

opt_fn = torch.compile(dynamic=False)(torch.matmul)
npu_out = opt_fn(input, weight)

Using /root/.cache/torch_extensions/py310_cu121 as PyTorch extensions root...
Emitting ninja build file /root/.cache/torch_extensions/py310_cu121/npu/build.ninja...
Building extension module npu...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module npu...


ninja: no work to do.
Wrapper Codegen Path = /tmp/torchinductor_root/yr/cyrl4zqohiglmrez32dmaijhd3sfdh4xabea5splhxwtwckiykpr.py
[Gem5] Gem5 is running..  
[Spike] Running Spike simulator
[TOGSim] TOGSim is running.   
[TOGSim] Simulation of "/root/workspace/PyTorchSim/tmp/4q4qv6gbpia/tile_graph.onnx" is stored to "/root/workspace/PyTorchSim/tmp/4q4qv6gbpia/togsim_result/2"


In [3]:
!cat /root/workspace/PyTorchSim/tmp/4q4qv6gbpia/togsim_result/0 | grep "Total execution cycle"

[2025-12-01 15:44:22.862] [info] Total execution cycles: 48536


### Manual Mapping
User can set tile size manually.

In [4]:
torch._dynamo.reset()

os.environ['TORCHSIM_MANUAL_TILE_SIZE']="1"
os.environ['TORCHSIM_TILE_M']="512"
os.environ['TORCHSIM_TILE_N']="512"
os.environ['TORCHSIM_TILE_K']="512"

input = torch.randn(1024, 1024).to(device=device)
weight = torch.randn(1024, 1024).to(device=device)

opt_fn = torch.compile(dynamic=False)(torch.matmul)
npu_out = opt_fn(input, weight)

Wrapper Codegen Path = /tmp/torchinductor_root/dg/cdgpndctsltydpvcgfyntgkgfmw6dp4a7vruwmjhxzilxzvyewby.py
[Gem5] Gem5 is running.   
[Spike] Running Spike simulator
[TOGSim] TOGSim is running... 
[TOGSim] Simulation of "/root/workspace/PyTorchSim/tmp/75hiq5mugpq/tile_graph.onnx" is stored to "/root/workspace/PyTorchSim/tmp/75hiq5mugpq/togsim_result/1"


In [5]:
!cat /root/workspace/PyTorchSim/tmp/75hiq5mugpq/togsim_result/0 | grep "Total execution cycle"

[2025-12-01 15:44:47.602] [info] Total execution cycles: 53419


### Autotune

In [6]:
torch._dynamo.reset()
os.environ['TORCHSIM_MANUAL_TILE_SIZE']="0"
os.environ['AUTOTUNE_TEMPLATE']="1"

input = torch.randn(1024, 1024).to(device=device)
weight = torch.randn(1024, 1024).to(device=device)

opt_fn = torch.compile(dynamic=False)(torch.matmul)
npu_out = opt_fn(input, weight)

Wrapper Codegen Path = /tmp/torchinductor_root/gg/cgg5mmgs4wa3u5od6mgxuausnbthyzfsnnauqtszfuq7qvl2qrj2.py
[Gem5] Gem5 is running..  
[Spike] Running Spike simulator
[TOGSim] TOGSim is running.   
[TOGSim] Simulation of "/root/workspace/PyTorchSim/tmp/4q4qv6gbpia/tile_graph.onnx" is stored to "/root/workspace/PyTorchSim/tmp/4q4qv6gbpia/togsim_result/3"


In [7]:
!cat /root/workspace/PyTorchSim/tmp/4q4qv6gbpia/togsim_result/1 | grep "Total execution cycle"

[2025-12-01 15:45:20.498] [info] Total execution cycles: 48564
